In [1]:
class Variable():

    ACROSS = "across"
    DOWN = "down"

    def __init__(self, i, j, direction, length):
        """Create a new variable with starting point, direction, and length."""
        self.i = i
        self.j = j
        self.direction = direction
        self.length = length
        self.cells = []
        for k in range(self.length):
            self.cells.append(
                (self.i + (k if self.direction == Variable.DOWN else 0),
                 self.j + (k if self.direction == Variable.ACROSS else 0))
            )

    def __hash__(self):
        return hash((self.i, self.j, self.direction, self.length))

    def __eq__(self, other):
        return (
            (self.i == other.i) and
            (self.j == other.j) and
            (self.direction == other.direction) and
            (self.length == other.length)
        )

    def __str__(self):
        return f"({self.i}, {self.j}) {self.direction} : {self.length}"

    def __repr__(self):
        direction = repr(self.direction)
        return f"Variable({self.i}, {self.j}, {direction}, {self.length})"


class Crossword():

    def __init__(self, structure_file, words_file):

        # Determine structure of crossword
        with open('data/structure0.txt') as f:
            contents = f.read().splitlines()
            self.height = len(contents)
            self.width = max(len(line) for line in contents)

            self.structure = []
            for i in range(self.height):
                row = []
                for j in range(self.width):
                    if j >= len(contents[i]):
                        row.append(False)
                    elif contents[i][j] == "_":
                        row.append(True)
                    else:
                        row.append(False)
                self.structure.append(row)

        # Save vocabulary list
        with open('data/words0.txt') as f:
            self.words = set(f.read().upper().splitlines())

        # Determine variable set
        self.variables = set()
        for i in range(self.height):
            for j in range(self.width):

                # Vertical words
                starts_word = (
                    self.structure[i][j]
                    and (i == 0 or not self.structure[i - 1][j])
                )
                if starts_word:
                    length = 1
                    for k in range(i + 1, self.height):
                        if self.structure[k][j]:
                            length += 1
                        else:
                            break
                    if length > 1:
                        self.variables.add(Variable(
                            i=i, j=j,
                            direction=Variable.DOWN,
                            length=length
                        ))

                # Horizontal words
                starts_word = (
                    self.structure[i][j]
                    and (j == 0 or not self.structure[i][j - 1])
                )
                if starts_word:
                    length = 1
                    for k in range(j + 1, self.width):
                        if self.structure[i][k]:
                            length += 1
                        else:
                            break
                    if length > 1:
                        self.variables.add(Variable(
                            i=i, j=j,
                            direction=Variable.ACROSS,
                            length=length
                        ))

        # Compute overlaps for each word
        # For any pair of variables v1, v2, their overlap is either:
        #    None, if the two variables do not overlap; or
        #    (i, j), where v1's ith character overlaps v2's jth character
        self.overlaps = dict()
        for v1 in self.variables:
            for v2 in self.variables:
                if v1 == v2:
                    continue
                cells1 = v1.cells
                cells2 = v2.cells
                intersection = set(cells1).intersection(cells2)
                if not intersection:
                    self.overlaps[v1, v2] = None
                else:
                    intersection = intersection.pop()
                    self.overlaps[v1, v2] = (
                        cells1.index(intersection),
                        cells2.index(intersection)
                    )

    def neighbors(self, var):
        """Given a variable, return set of overlapping variables."""
        return set(
            v for v in self.variables
            if v != var and self.overlaps[v, var]
        )

In [2]:
import sys

class CrosswordCreator():

    def __init__(self, crossword):
        """
        Create new CSP crossword generate.
        """
        self.crossword = crossword
        self.domains = {
            var: self.crossword.words.copy()
            for var in self.crossword.variables
        }

    def letter_grid(self, assignment):
        """
        Return 2D array representing a given assignment.
        """
        letters = [
            [None for _ in range(self.crossword.width)]
            for _ in range(self.crossword.height)
        ]
        for variable, word in assignment.items():
            direction = variable.direction
            for k in range(len(word)):
                i = variable.i + (k if direction == Variable.DOWN else 0)
                j = variable.j + (k if direction == Variable.ACROSS else 0)
                letters[i][j] = word[k]
        return letters

    def print(self, assignment):
        """
        Print crossword assignment to the terminal.
        """
        letters = self.letter_grid(assignment)
        for i in range(self.crossword.height):
            for j in range(self.crossword.width):
                if self.crossword.structure[i][j]:
                    print(letters[i][j] or " ", end="")
                else:
                    print("█", end="")
            print()

    def save(self, assignment, filename):
        """
        Save crossword assignment to an image file.
        """
        from PIL import Image, ImageDraw, ImageFont
        cell_size = 100
        cell_border = 2
        interior_size = cell_size - 2 * cell_border
        letters = self.letter_grid(assignment)

        # Create a blank canvas
        img = Image.new(
            "RGBA",
            (self.crossword.width * cell_size,
             self.crossword.height * cell_size),
            "black"
        )
        font = ImageFont.truetype("assets/fonts/OpenSans-Regular.ttf", 80)
        draw = ImageDraw.Draw(img)

        for i in range(self.crossword.height):
            for j in range(self.crossword.width):

                rect = [
                    (j * cell_size + cell_border,
                     i * cell_size + cell_border),
                    ((j + 1) * cell_size - cell_border,
                     (i + 1) * cell_size - cell_border)
                ]
                if self.crossword.structure[i][j]:
                    draw.rectangle(rect, fill="white")
                    if letters[i][j]:
                        w, h = draw.textsize(letters[i][j], font=font)
                        draw.text(
                            (rect[0][0] + ((interior_size - w) / 2),
                             rect[0][1] + ((interior_size - h) / 2) - 10),
                            letters[i][j], fill="black", font=font
                        )

        img.save(filename)

    def solve(self):
        """
        Enforce node and arc consistency, and then solve the CSP.
        """
        self.enforce_node_consistency()
        self.ac3()
        return self.backtrack(dict())
    
    def index_in_list(n, nlist): # help function 1
        '''find a index of the word in the list of words'''
        l = []
        for index, value in enumerate(nlist):
            if value == n:
                l.append(index)
        return l

    def remove_words_not_consistent(d, ndict): # help fuction 2
        '''return only words that in index list'''
        l = set()
        for index, value in enumerate(ndict):
            if index in d:
                l.add(value)
        return l

    def enforce_node_consistency(self):
        """
        Update `self.domains` such that each variable is node-consistent.
        (Remove any values that are inconsistent with a variable's unary
         constraints; in this case, the length of the word.)
        """
        # get the list of leght of words 
        for i in test.values():
            lenth_of_words = [len(d) for d in i]
            break # count one time
        
        # iterate over dict items 
        for i, values in test.items():
        #     print(index_in_list(i.length, lenth_of_words))
        #     print(test[i])
        #     print(remove_words_not_consistent(index_in_list(i.length, lenth_of_words), test[i]))
        
            # update dict of the domains
            # find index in the list that not consistent with task
            # check lenght of the word in assigment and update with new values
            test[i] = remove_words_not_consistent(index_in_list(i.length, lenth_of_words), test[i])


    def revise(self, x, y):
        """
        Make variable `x` arc consistent with variable `y`.
        To do so, remove values from `self.domains[x]` for which there is no
        possible corresponding value for `y` in `self.domains[y]`.
        Return True if a revision was made to the domain of `x`; return
        False if no revision was made.
        """
        # start with false 
        revised = False
        # empty list
        update_words = []
        # check if words overlap in assigment
        if self.crossword.overlaps[x, y] is not None:
            # iterate over x and y values
            for word1 in self.domains[x]:
                for word2 in self.domains[y]:
                    # 1 words should not be the same
                    # 2 the lengh of them must be more that letters of overlap
                    # 3 check if i-th value of x not equal to the j-th value of y
                    if word1 != word2 \
                    and len(word1) >= (self.overlaps[x, y][0] + 1)\
                    and len(word2) >= (self.overlaps[x, y][1] + 1) \
                    and word1[self.overlaps[x, y][0]] != word2[self.overlaps[x, y][1]]:
                        # change status of revised
                        revised = True
                        # add to the list
                        update_words.append(word1)
        # itetare of the NOT acr consistem words of x and remome them
        for word in set(update_words):
            # pop the values of x from domain
            self.domains[x].remove(word)
        #return the status 
        return revised

    def ac3(self, arcs=None):
        """
        Update `self.domains` such that each variable is arc consistent.
        If `arcs` is None, begin with initial list of all arcs in the problem.
        Otherwise, use `arcs` as the initial list of arcs to make consistent.

        Return True if arc consistency is enforced and no domains are empty;
        return False if one or more domains end up empty.
        """
        raise NotImplementedError

    def assignment_complete(self, assignment):
        """
        Return True if `assignment` is complete (i.e., assigns a value to each
        crossword variable); return False otherwise.
        """
        raise NotImplementedError

    def consistent(self, assignment):
        """
        Return True if `assignment` is consistent (i.e., words fit in crossword
        puzzle without conflicting characters); return False otherwise.
        """
        raise NotImplementedError

    def order_domain_values(self, var, assignment):
        """
        Return a list of values in the domain of `var`, in order by
        the number of values they rule out for neighboring variables.
        The first value in the list, for example, should be the one
        that rules out the fewest values among the neighbors of `var`.
        """
        raise NotImplementedError

    def select_unassigned_variable(self, assignment):
        """
        Return an unassigned variable not already part of `assignment`.
        Choose the variable with the minimum number of remaining values
        in its domain. If there is a tie, choose the variable with the highest
        degree. If there is a tie, any of the tied variables are acceptable
        return values.
        """
        raise NotImplementedError

    def backtrack(self, assignment):
        """
        Using Backtracking Search, take as input a partial assignment for the
        crossword and return a complete assignment if possible to do so.

        `assignment` is a mapping from variables (keys) to words (values).

        If no assignment is possible, return None.
        """
        raise NotImplementedError


# def main():

#     # Check usage
if len(sys.argv) not in [3, 4]:
    sys.exit("Usage: python generate.py structure words [output]")

# Parse command-line arguments
structure = sys.argv[1]
words = sys.argv[2]
output = sys.argv[3] if len(sys.argv) == 4 else None

    # Generate crossword
crossword = Crossword(structure, words)
creator = CrosswordCreator(crossword)
    # assignment = creator.solve()

    # # Print result
    # if assignment is None:
    #     print("No solution.")
    # else:
    #     creator.print(assignment)
    #     if output:
    #         creator.save(assignment, output)


In [3]:
crossword.overlaps, crossword.words, crossword.variables

({(Variable(0, 1, 'down', 5), Variable(4, 1, 'across', 4)): (4, 0),
  (Variable(0, 1, 'down', 5), Variable(1, 4, 'down', 4)): None,
  (Variable(0, 1, 'down', 5), Variable(0, 1, 'across', 3)): (0, 0),
  (Variable(4, 1, 'across', 4), Variable(0, 1, 'down', 5)): (0, 4),
  (Variable(4, 1, 'across', 4), Variable(1, 4, 'down', 4)): (3, 3),
  (Variable(4, 1, 'across', 4), Variable(0, 1, 'across', 3)): None,
  (Variable(1, 4, 'down', 4), Variable(0, 1, 'down', 5)): None,
  (Variable(1, 4, 'down', 4), Variable(4, 1, 'across', 4)): (3, 3),
  (Variable(1, 4, 'down', 4), Variable(0, 1, 'across', 3)): None,
  (Variable(0, 1, 'across', 3), Variable(0, 1, 'down', 5)): (0, 0),
  (Variable(0, 1, 'across', 3), Variable(4, 1, 'across', 4)): None,
  (Variable(0, 1, 'across', 3), Variable(1, 4, 'down', 4)): None},
 {'EIGHT',
  'FIVE',
  'FOUR',
  'NINE',
  'ONE',
  'SEVEN',
  'SIX',
  'TEN',
  'THREE',
  'TWO'},
 {Variable(0, 1, 'across', 3),
  Variable(0, 1, 'down', 5),
  Variable(1, 4, 'down', 4),
  Vari

In [4]:
crossword.height, crossword.structure, crossword.variables, crossword.width

(5,
 [[False, True, True, True, False],
  [False, True, False, False, True],
  [False, True, False, False, True],
  [False, True, False, False, True],
  [False, True, True, True, True]],
 {Variable(0, 1, 'across', 3),
  Variable(0, 1, 'down', 5),
  Variable(1, 4, 'down', 4),
  Variable(4, 1, 'across', 4)},
 5)

In [5]:
def revise(self, x, y):
    """
    Make variable `x` arc consistent with variable `y`.
    To do so, remove values from `self.domains[x]` for which there is no
    possible corresponding value for `y` in `self.domains[y]`.
    Return True if a revision was made to the domain of `x`; return
    False if no revision was made.
    """
    # start with false 
    revised = False
    # empty list
    update_words = []
    # check if words overlap in assigment
    if crossword.overlaps[x, y] is not None:
        # iterate over x and y values
        for word1 in creator.domains[x]:
            for word2 in creator.domains[y]:
                # 1 words should not be the same
                # 2 the lengh of them must be more that letters of overlap
                # 3 check if i-th value of x not equal to the j-th value of y
                if word1 != word2 \
                and len(word1) >= (crossword.overlaps[x, y][0] + 1)\
                and len(word2) >= (crossword.overlaps[x, y][1] + 1) \
                and word1[crossword.overlaps[x, y][0]] != word2[crossword.overlaps[x, y][1]]:
                    # change status of revised
                    revised = True
                    # add to the list
                    update_words.append(word1)
    # itetare of the NOT acr consistem words of x and remome them
    for word in set(update_words):
        # pop the values of x from domain
        creator.domains[x].remove(word)
    #return the status 
    return revised, update_words
 
# revise(None, Variable(1, 4, 'down', 4), Variable(0, 1, 'across', 3))

revise(None, Variable(1, 4, 'down', 4), Variable(4, 1, 'across', 4))

(True,
 ['FIVE',
  'FIVE',
  'THREE',
  'THREE',
  'NINE',
  'NINE',
  'EIGHT',
  'EIGHT',
  'EIGHT',
  'EIGHT',
  'EIGHT',
  'SEVEN',
  'SEVEN',
  'FOUR',
  'FOUR',
  'FOUR',
  'FOUR',
  'FOUR'])

In [6]:
crossword.overlaps[(Variable(1, 4, 'down', 4), Variable(4, 1, 'across', 4))]

(3, 3)

In [7]:
creator.domains

{Variable(0, 1, 'down', 5): {'EIGHT',
  'FIVE',
  'FOUR',
  'NINE',
  'ONE',
  'SEVEN',
  'SIX',
  'TEN',
  'THREE',
  'TWO'},
 Variable(4, 1, 'across', 4): {'EIGHT',
  'FIVE',
  'FOUR',
  'NINE',
  'ONE',
  'SEVEN',
  'SIX',
  'TEN',
  'THREE',
  'TWO'},
 Variable(1, 4, 'down', 4): {'ONE', 'SIX', 'TEN', 'TWO'},
 Variable(0, 1, 'across', 3): {'EIGHT',
  'FIVE',
  'FOUR',
  'NINE',
  'ONE',
  'SEVEN',
  'SIX',
  'TEN',
  'THREE',
  'TWO'}}

In [8]:
crossword.overlaps

{(Variable(0, 1, 'down', 5), Variable(4, 1, 'across', 4)): (4, 0),
 (Variable(0, 1, 'down', 5), Variable(1, 4, 'down', 4)): None,
 (Variable(0, 1, 'down', 5), Variable(0, 1, 'across', 3)): (0, 0),
 (Variable(4, 1, 'across', 4), Variable(0, 1, 'down', 5)): (0, 4),
 (Variable(4, 1, 'across', 4), Variable(1, 4, 'down', 4)): (3, 3),
 (Variable(4, 1, 'across', 4), Variable(0, 1, 'across', 3)): None,
 (Variable(1, 4, 'down', 4), Variable(0, 1, 'down', 5)): None,
 (Variable(1, 4, 'down', 4), Variable(4, 1, 'across', 4)): (3, 3),
 (Variable(1, 4, 'down', 4), Variable(0, 1, 'across', 3)): None,
 (Variable(0, 1, 'across', 3), Variable(0, 1, 'down', 5)): (0, 0),
 (Variable(0, 1, 'across', 3), Variable(4, 1, 'across', 4)): None,
 (Variable(0, 1, 'across', 3), Variable(1, 4, 'down', 4)): None}